# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [1]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [2]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [8]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = "2023-10-11"
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2835,2023-10-06,"$27,412.12","$28,252.54","$27,215.55","$27,946.60","$27,946.60",13492391599
2836,2023-10-07,"$27,946.78","$28,028.09","$27,870.42","$27,968.84","$27,968.84",6553044316
2837,2023-10-08,"$27,971.68","$28,102.17","$27,740.66","$27,935.09","$27,935.09",7916875290
2838,2023-10-09,"$27,934.47","$27,989.47","$27,302.56","$27,583.68","$27,583.68",12007668568
2839,2023-10-10,"$27,589.20","$27,715.85","$27,301.65","$27,391.02","$27,391.02",9973350678


In [4]:
# TAREA: Validamos que no hay datos vacíos
btc_df.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [9]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[["Date","Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
new_names = {
    "Date": "ds",
    "Open": "y",
}

# Los renombramos con los nuevos nombres
df.rename(columns=new_names, inplace=True)


In [11]:
df

,ds,y
0,2016-01-01,$430.72
1,2016-01-02,$434.62
2,2016-01-03,$433.58
3,2016-01-04,$430.06
4,2016-01-05,$433.07
...,...,...
2835,2023-10-06,"$27,412.12"
2836,2023-10-07,"$27,946.78"
2837,2023-10-08,"$27,971.68"
2838,2023-10-09,"$27,934.47"


In [12]:
df['ds'] = df['ds'].dt.tz_localize(None)

In [13]:
df.tail()

,ds,y
2835,2023-10-06,"$27,412.12"
2836,2023-10-07,"$27,946.78"
2837,2023-10-08,"$27,971.68"
2838,2023-10-09,"$27,934.47"
2839,2023-10-10,"$27,589.20"


In [14]:
# Crear la gráfica del precio de apertura

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text="Serie temporal de Bitcoin Precio de Apertura",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



Documentacion oficial de Prophet [GitHub](https://facebook.github.io/prophet/docs/quick_start.html)

In [15]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode="multiplicative")

In [16]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpx3t4446u/6a9q7cjn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpx3t4446u/jy8q3nr5.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=54619', 'data', 'file=/tmp/tmpx3t4446u/6a9q7cjn.json', 'init=/tmp/tmpx3t4446u/jy8q3nr5.json', 'output', 'file=/tmp/tmpx3t4446u/prophet_modelpicwigo8/prophet_model-20231011084236.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:42:36 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:42:36 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [18]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
3200,2024-10-05
3201,2024-10-06
3202,2024-10-07
3203,2024-10-08
3204,2024-10-09


In [19]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
...,...
3200,2024-10-05
3201,2024-10-06
3202,2024-10-07
3203,2024-10-08


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [20]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [21]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3200,2024-10-05,"$17,103.91","$-1,267.95","$33,842.07"
3201,2024-10-06,"$17,292.94","$-2,340.89","$34,885.47"
3202,2024-10-07,"$17,500.39","$-2,188.10","$35,284.65"
3203,2024-10-08,"$17,680.20","$-1,750.49","$34,921.42"
3204,2024-10-09,"$17,851.22","$-1,847.30","$35,222.68"


In [22]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

21743.395586357492

<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [23]:
plot_plotly(m, forecast)

In [24]:
plot_components_plotly(m, forecast)